### Query execution plans

**How to View the Plan**
- In the Databricks SQL Editor, after you run a query, click the "Query Profile" tab at the bottom. This opens a graphical representation of the execution plan.

**1. The "Life of a Query" (Plan Stages)**
- When you hit "Run," the SQL Warehouse goes through three main steps:
- **Parsing/Analysis:** Checks if your table actually exists and if you have permissions.
- **Optimization (The CBO):** The Cost-Based Optimizer looks at table statistics (size, file count) and decides the best way to join tables.
- **Physical Plan:** The actual Spark/Photon code that runs on the CPUs.

**2. What to Look for in the Query Profile**
- When analyzing queries, look for these specific "Operations":
- **FileScan (Delta):** This is where it reads data from S3/ADLS. Look at "Data Read" vs. "Data Skipped." If your query skips 40M rows and only reads 2M, your partitioning is working perfectly.
- **PhotonGate:** If you see this, it means the Photon Engine has taken over. This is good—it means your query is running in highly optimized C++.
- **Shuffle (Exchange):** This is the "expensive" part where data moves between nodes. If you see massive shuffles, your JOIN or GROUP BY might be inefficient.
- **Broadcast Hash Join:** This is the "Gold Standard" for performance. It means one table was small enough to be sent to all workers, avoiding a slow shuffle.

**3. Optimization Techniques**
- If the execution plan shows your query is slow, here are three ways to fix it without changing the code:
- **ANALYZE TABLE:** Run ANALYZE TABLE table COMPUTE STATISTICS;. This tells the optimizer exactly how big the table is so it can pick the best plan.
- **Z-ORDERING:** If you often filter by brand, run OPTIMIZE table ZORDER BY (column);. This physically rearranges the rows so the "FileScan" can skip irrelevant data.
- **Data Type Alignment:** Ensure you aren't joining a STRING ID to an INT ID. This forces a "Cast," which can break "Predicate Pushdown" (the ability to skip data).

### Partitioning strategies

**1. The Core Concept: "Data Skipping"**
- Partitioning physically organizes your data into folders based on a specific column. 
- When you run a query with a WHERE clause on that column, Databricks "skips" every folder that doesn't match, meaning it never even reads those files.

**2. Choosing the Right Column**
- For e-commerce project, you have three main strategies:
**A. Partitioning by Date (event_date)**
- Why: Most e-commerce queries ask for "Last 7 days" or "This month."
- Performance: If you query 1 day of data, the Warehouse only reads ~1/30th of your monthly data.
- Implementation:
```
Python
df.write.partitionBy("event_date").saveAsTable("...")
```
**B. The "High Cardinality" Trap**
- What to Avoid: Never partition by a column with too many unique values (like user_id or product_id).
- The Result: This creates thousands of tiny files ("The Small File Problem"), which actually makes the Starter Warehouse slower because of the overhead of opening so many files.
**C. Z-Ordering (The Modern Alternative)**
- If you need to filter by brand or category frequently, don't partition by them. Instead, use Z-Ordering. It's like a multidimensional index that co-locates related data within the existing files.
- Implementation:
```
SQL
OPTIMIZE ecommerce_prod.silver.cleaned_events 
ZORDER BY (brand, main_category);
```

**3. Updated Strategy: Liquid Clustering**
- As of 2024-2025, Databricks has introduced Liquid Clustering, which replaces traditional partitioning. It is much smarter for Serverless environments because it re-balances itself as your data grows.
- If you were to rewrite your Silver table today, you would use this:
```
SQL
CREATE TABLE ecommerce_prod.silver.cleaned_events
CLUSTER BY (event_date, brand)
AS SELECT * FROM bronze_data;
```
- Benefit: You don't have to worry about "over-partitioning." The Starter Warehouse handles the file layout automatically.

**4. How to check if your partitioning is working**
- After running a query in your SQL Warehouse, check the Query Profile:
- Look for the FileScan node.
- Check the metric: "files skipped" vs. "files read."
- If "files skipped" is a large number, your partitioning strategy is a success!

### OPTIMIZE & ZORDER

- While Partitioning creates physical folders, OPTIMIZE and Z-ORDER work inside those folders to make data access even faster. 
- This is especially critical for a Serverless Starter Warehouse because it reduces the amount of data the CPU has to process.

**1. OPTIMIZE:** The "Data Cleaner"
- Over time, as you add data to a table, Delta Lake creates many small files. 
- Reading 1,000 tiny files is much slower than reading one large 1GB file.
- **What it does:** It performs file compaction. It takes all those small, fragmented files and merges them into larger, more efficient files (usually ~1GB).
- **When to use it:** Run it after a large data load or if your dashboard queries start slowing down.
```
SQL
OPTIMIZE global_flights;
```

**2. Z-ORDER: The "Internal Organizer"**
- Z-Ordering is a technique used to co-locate related information in the same set of files. 
- Think of it like organizing a bookshelf not just by the author's name (Partitioning), but also by the color of the book spine within that section.
- **What it does:** It maps multiple columns into a "space-filling curve." This ensures that data with similar values for the Z-Ordered columns are physically stored together.
- **The Benefit:** When you filter by a Z-Ordered column, the engine can skip entire files within a partition that don't contain your data.

**3. Difference Between Partitioning & Z-Ordering**
- It is common to use both together, but they serve different purposes:

| Feature | Partitioning | Z-Ordering |
| ----- | ----- | ----- |
| Physical Structure | Creates separate folders. | Reorganizes data within files.
| Best For | Low-cardinality columns (e.g., Year, Country). | High-cardinality columns (e.g., City, Brand).|
| Query Benefit | Skips entire folders (Partition Pruning). | Skips files within folders (Data Skipping).
| Maintenance | Set once during table creation. | Should be run periodically as data grows.

**4. Practical Example (Aviation Table)**
- Imagine you have a table partitioned by Year, but your users always filter by Departure_City.
- The Query:
```
SQL
SELECT * FROM global_flights 
WHERE Year = 2024 
AND Departure_City = 'London';
```
- The Optimization Strategy:
- Partition by Year: The engine goes straight to the 2024 folder.
- Z-Order by Departure_City: Inside the 2024 folder, the engine only opens the 2-3 files that contain 'London' data, skipping the hundreds of other files containing 'Paris' or 'New York'.
- The Command:
```
SQL 
OPTIMIZE global_flights 
ZORDER BY (Departure_City);
```

**5. Modern Alternative: Liquid Clustering**
If you are using the latest version of Databricks (which the Serverless Warehouse supports), you can replace both Partitioning and Z-Ordering with Liquid Clustering. It handles all of this automatically without you needing to run OPTIMIZE manually.
```
SQL
CREATE TABLE global_flights
CLUSTER BY (Year, Departure_City);
```

### Caching techniques

- In a Serverless Starter Warehouse, caching is your best defense against performance bottlenecks. 
- Because you have a fixed amount of compute, caching ensures that the warehouse doesn't have to re-calculate the same data every time a user refreshes your dashboard.
- Databricks uses a three-layered caching strategy to make queries near-instant.

**1. Delta Cache (Local SSD Cache)**
- This is the most important cache for performance. When you query data from cloud storage (S3/ADLS), the Warehouse stores a copy of that data on the local SSDs of the compute nodes.
- **How it works:** The data is stored in an uncompressed, high-speed format.
- **The Benefit:** If you run a query for "Top Products" and then run it again 5 minutes later, the Warehouse pulls the data from the SSD (microseconds) instead of the cloud storage (milliseconds).
- **Maintenance:** On Serverless Warehouses, this is managed automatically. If a node is shut down, the cache is cleared.

**2. Query Result Cache**
- This cache sits at the "Top" level. It doesn't store the data rows; it stores the final answer to your SQL query.
- **How it works:** If you run SELECT sum(revenue) FROM sales, and the underlying table hasn't changed, Databricks simply hands you the result from the previous run.
- **The Benefit:** It bypasses the entire execution engine (no FileScan, no Shuffle). Results appear in milliseconds.
- **Visibility:** You can see this in the Query Profile. It will explicitly state: "Result served from cache."

**3. Spark Cache (Memory Cache)**
- This is a manual cache often used in Notebooks (PySpark), though less common in SQL Warehouses.
- **How it works:** You explicitly tell Spark to keep a specific DataFrame in the RAM (Memory).
- **The Command:** df.cache() or df.persist().
- **Difference:** Unlike the Delta Cache (which is SSD-based), this is RAM-based. It is faster but much more expensive in terms of resources.

- Comparison of Caching Layers

| Cache Type | Stored In | Content | Best For | 
| ----- | ----- | ----- | ----- |
| Result Cache | Warehouse Manager | Final Query Output |Repeated Dashboard refreshes.
| Delta Cache | Local SSD | Raw Data Blocks | Speeding up different queries on the same table.
| Spark Cache | Worker RAM | Processed DataFrames | Iterative Machine Learning or complex joins.

### Task 1: Analyze Query Plans

In [0]:
%sql
-- Baseline Query: Checking revenue by brand
-- Look at the 'Query Profile' to see if it's reading all 42M rows (FileScan)
SELECT 
  brand, 
  SUM(price) as total_revenue
FROM ecommerce_prod.silver.cleaned_events
WHERE event_type = 'purchase'
GROUP BY 1
ORDER BY 2 DESC;

### Task 2: Partition Large Tables

In [0]:
%sql
CREATE OR REPLACE TABLE ecommerce_prod.silver.cleaned_events
PARTITIONED BY (event_date)
AS 
SELECT 
  *, 
  -- Cast to timestamp first, then extract the date to ensure no NULLs
  to_date(CAST(event_time AS TIMESTAMP)) AS event_date 
FROM ecommerce_prod.bronze.raw_events
-- Ensure we only bring over rows where the date conversion actually worked
WHERE event_time IS NOT NULL;

In [0]:
%sql
SELECT 
  event_time, 
  typeof(event_time) as data_type,
  brand
FROM ecommerce_prod.bronze.raw_events 
LIMIT 10;

### Task 3: Apply ZORDER

In [0]:
%sql
CREATE OR REPLACE TABLE ecommerce_prod.silver.cleaned_events
PARTITIONED BY (event_date)
AS 
SELECT 
  *, 
  to_date(CAST(event_time AS TIMESTAMP)) AS event_date,
  -- Extract the first part of category_code (e.g., 'electronics' from 'electronics.video.tv')
  split(category_code, '\\.')[0] AS main_category 
FROM ecommerce_prod.bronze.raw_events;

In [0]:
%sql
SELECT count(*) FROM ecommerce_prod.silver.cleaned_events;

### Task 4: Benchmark Improvements

In [0]:
%sql
-- Task 4: Optimized Benchmark
SELECT 
  main_category,
  SUM(price) as category_revenue
FROM ecommerce_prod.silver.cleaned_events
WHERE event_date = '2019-11-01'  -- Use a date found in Step 1
  AND brand = 'samsung'          -- Use a brand found in Step 1 (match the casing!)
GROUP BY 1;